In [15]:
import pdfminer.high_level
import os
import os.path
import pandas as pd


In [16]:
data_dir = os.path.join('..', 'Data Exploration - AC')
file = os.path.join(data_dir, '1990_april_5_790191468211457471_conformed-copy--l3076--west-mitidja-irrigation-project--loan-agreement.pdf')

In [17]:
print(file)

../Data Exploration - AC/1990_april_5_790191468211457471_conformed-copy--l3076--west-mitidja-irrigation-project--loan-agreement.pdf


In [18]:
raw_parse = pdfminer.high_level.extract_text(file)
print(raw_parse)

d
e
z
i
r
o
h
t
u
A
 
e
r
u
s
o
c
s
D
 
c

i

l

i
l

b
u
P

d
e
z
i
r
o
h
t
u
A
 
e
r
u
s
o
c
s
D
 
c

i

l

i
l

b
u
P

d
e
z
i
r
o
h

t

 

u
A
e
r
u
s
o
c
s
D
 
c

l

i

i
l

b
u
P

d
e
z
i
r
o
h
t
u
A
 
e
r
u
s
o
c
s
D
 
c

i

l

i
l

b
u
P

CONFORMED COPY 

LOAN NUMBER 3076 AL

Loan Agreement

(West Mitidja Irrigation Project)

DEMOCRATIC AND POPULAR REPUBLIC OF ALGERIA

between

and

INTERNATIONAL BANK FOR RECONSTRUCTION
AND DEVELOPMENT

Dated April 5, 1990

LOAN AGREEMENT                     

LOAN NUMBER 3076 AL

     AGREEMENT, dated April 5, 1990, between DEMOCRATIC and POPULAR REPUBLIC OF 
ALGERIA (the Borrower) and INTERNATIONAL BANK FOR RECONSTRUCTION AND DEVELOPMENT (the 
Bank).

     WHEREAS the Borrower, having satisfied itself as to the feasibility and priority 
of the Project described in Schedule 2 to this Agreement, has requested the Bank to 
assist in the financing of the Project; and

     WHEREAS the Bank has agreed, on the basis, inter alia, of the foregoing, t

In [32]:
raw_parse = raw_parse.strip('d\ne\nz\ni\nr\no\nh\nt\nu\nA\n \ne\nr\nu\ns\no\nc\ns\nD\n \nc\n\nl\n\ni\n\ni\nl\n\nb\nu\nP\n\nd\ne\nz\ni\nr\no\nh\nt\nu\nA\n \ne\nr\nu\ns\no\nc\ns\nD\n \nc\n\ni\n\nl\n\ni\nl\n\nb\nu\nP\n\nd\ne\nz\ni\nr\no\nh\n\nt\n\n \n\nu\nA\ne\nr\nu\ns\no\nc\ns\nD\n \nc\n\nl\n\ni\n\ni\nl\n\nb\nu\nP\n\nd\ne\nz\ni\nr\no\nh\nt\nu\nA\n \ne\nr\nu\ns\no\nc\ns\nD\n \nc\n\ni\n\nl\n\ni\nl\n\nb\nu\nP\n\n')

In [28]:
import re

#The following function formats the text to be able to differentiate paragraphs and lines. 
#Declare currency characters, to identify amount of loan. 
currency_characters=u'$¢£¤¥֏؋৲৳৻૱௹฿៛\u20a0-\u20bd\ua838\ufdfc\ufe69\uff04\uffe0\uffe1\uffe5\uffe6'

def separate_lines(text,currency_characters=currency_characters,simple_line=' ',separator='\n'):  
    #Treat dot, colon and semicolon followed by newline as a new paragraph. 
    text=re.sub('[:;.](\n)','\n\n',text)
    
    #Eliminate lines of only space characters
    text=re.sub('\n\s+\n','\n\n',text)
    
    #Replace single newlines as an space (since we are still in the same paragraph)
    text=re.sub('\n(?!\n)',simple_line,text)
    
    #Replace multiple newlines with a single one. 
    text=re.sub('\n+[\n\s]*','\n',text)
    
    #Erase non alphanumeric, dots, parenthesis or currency characters
    re_string='[^0-9a-zA-Z\s.\(\)'+currency_characters+']+'
    text=re.sub(re_string, '',text)
    text=text.replace('"','')
    text=re.sub('( ){2,}',' ',text)
    
    #Separate into lines/paragraphs
    lines=re.split(separator,text)
    lines=[l.strip() for l in lines]
    return lines

In [50]:
raw_parse

['CONFORMED COPY',
 'LOAN NUMBER 3076 AL',
 'Loan Agreement',
 '(West Mitidja Irrigation Project)',
 'DEMOCRATIC AND POPULAR REPUBLIC OF ALGERIA',
 'between',
 'and',
 'INTERNATIONAL BANK FOR RECONSTRUCTION AND DEVELOPMENT',
 'Dated April 5 1990',
 'LOAN AGREEMENT',
 'LOAN NUMBER 3076 AL',
 'AGREEMENT dated April 5 1990 between DEMOCRATIC and POPULAR REPUBLIC OF ALGERIA (the Borrower) and INTERNATIONAL BANK FOR RECONSTRUCTION AND DEVELOPMENT (the Bank)',
 'WHEREAS the Borrower having satisfied itself as to the feasibility and priority of the Project described in Schedule 2 to this Agreement has requested the Bank to assist in the financing of the Project and',
 'WHEREAS the Bank has agreed on the basis inter alia of the foregoing to extend the Loan to the Borrower upon the terms conditions set forth in this Agreement',
 'NOW THEREFORE the parties hereto hereby agree as follows',
 'ARTICLE I',
 'General Conditions Definitions',
 'Section 1.01. The General Conditions Applicable to Loan a

In [34]:
def extract_parenthesis(string):
    flag = 0
    result, accum = [], []
    for c in string:
        if c == ')':
            flag -= 1
        if flag:
            accum.append(c)
        if c == '(':
            flag += 1
        if not flag and accum:
            result.append(''.join(accum))
            accum = []
    return result

In [36]:
unwanted_words=['borrower','lender','rate','guarantee','agreement','guarantor','amendment']
max_line_first_page=35 #This was decided after trying some values

#Returns: Project Name, line number where it was found.
def get_project_name(lines,line_num,unwanted_words=unwanted_words,max_line_first_page=max_line_first_page):
    found_start=False
    line_max=min([max_line_first_page,len(lines)]) 
    while line_num <line_max:
        l=lines[line_num]
        if found_start:
            #Look for a line of text in between two parenthesis
            m=re.search('\(\s*([a-z]+[(),:;a-z\s\d]+)(project)?\s*\)',l.lower())
            if m!=None and len(m.group(1))>12:
                #Sometimes Parenthesis can be messy, so we extract the text on the outermost parenthesis
                if len(re.findall('\(',m.group(0)))>1:
                    name=extract_parenthesis(m.group(0))
                    if len(name)>0: #This must have at least one string if the parenthesis were balanced
                        name=name[0]
                    else: #if the parenthesis were not balanced, return everything before the second (
                        m=re.search('(.*)\(',m.group(1))
                        name=m.group(1)
                else:
                    name=m.group(1)
                if len(name)>12 and not(any(elem in name.split(' ') for elem in unwanted_words)) and not ('general conditions' in name):
                    return name,line_num
        else:
            words=l.lower().split(' ')
            if ('loan' in words) or ('agreement' in words):
                #Once we found the sentence with loan or agreement in it, we start looking for the Project Name
                found_start=True
                line_num-=1
        line_num+=1
    #If couldnt find a Project Name following the format, returns None
    return None,0

In [58]:
get_project_name(raw_parse, 0) 

('west mitidja irrigation project', 3)